# HUẤN LUYỆN MÔ HÌNH SVM
Thực hiện huấn luyện mô hình SVM dựa trên các đặc trưng đã được trích suất từ ANN để phát hiện giao dịch gian lận.


#### Khai báo thư viện

In [1]:
# Import các thư viện
import os
import pandas as pd
import numpy as np
from keras import Model
from keras.src.saving import load_model
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import joblib
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, auc

2024-12-04 00:56:43.585749: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/nguyendinhkhai/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


#### Khai báo đường dẫn

In [2]:
# Đường dẫn dữ liệu và mô hình
split_dir = "../data/old_data/splits"
model_dir = "../models"
svm_model_path = f"{model_dir}/svm_model.pkl"
plot_dir = "../plots"

### Tải dữ liệu
Dữ liệu train và test được đọc từ các file đã chia.


In [3]:
# Đọc dữ liệu
X_train = pd.read_csv(f"{split_dir}/X_train.csv")
y_train = pd.read_csv(f"{split_dir}/y_train.csv")
X_test = pd.read_csv(f"{split_dir}/X_test.csv")
y_test = pd.read_csv(f"{split_dir}/y_test.csv")

# Kiểm tra dữ liệu
print(f"Kích thước tập train: {X_train.shape}")
print(f"Kích thước tập test: {X_test.shape}")


Kích thước tập train: (742826, 8)
Kích thước tập test: (93791, 8)


### Tải và trích xuất đặc trưng từ ANN

In [4]:
# Tải mô hình ANN đã huấn luyện
ann_model = load_model(f"{model_dir}/ann_model.keras")

# Xem kiến trúc mô hình đã tải
ann_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,605 (37.52 KB)

 Trainable params: 3,201 (12.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,404 (25.02 KB)

In [5]:
feature_extractor = Model(inputs=ann_model.inputs, outputs=ann_model.layers[-2].output)

# Trích xuất đặc trưng
X_train_features = feature_extractor.predict(X_train)
X_test_features = feature_extractor.predict(X_test)

print(f"Kích thước đặc trưng tập train: {X_train_features.shape}")
print(f"Kích thước đặc trưng tập test: {X_test_features.shape}")


23214/23214 ━━━━━━━━━━━━━━━━━━━━ 22s 949us/step
2931/2931 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
Kích thước đặc trưng tập train: (742826, 16)
Kích thước đặc trưng tập test: (93791, 16)


### Huấn luyện SVM
Mô hình SVM được cấu hình với: Kernel: Linear.


In [ ]:
# Khởi tạo và huấn luyện SVM
svm_model = SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)
svm_model.fit(X_train_features, y_train.values.ravel())


#### Biểu đồ phân tách của SVM

In [ ]:
# Giảm chiều xuống 2 cho mục đích trực quan hóa
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_features)
X_test_pca = pca.transform(X_test_features)  # Tương tự cho tập test


ValueError: n_components=2 must be between 0 and min(n_samples, n_features)=1 with svd_solver='covariance_eigh'

In [ ]:
# Sử dụng mô hình đã huấn luyện để dự đoán
y_train_pred = svm_model.predict(X_train_pca)

# Vẽ dữ liệu
plt.figure(figsize=(10, 6))
scatter = plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, cmap='coolwarm', s=20, label="Dữ liệu gốc")
plt.title("Biểu Đồ Phân Tách Dữ Liệu Với SVM")
plt.xlabel("Không gian lận")
plt.ylabel("Gian lận")

# Vẽ ranh giới phân tách
x_min, x_max = X_train_pca[:, 0].min() - 1, X_train_pca[:, 0].max() + 1
y_min, y_max = X_train_pca[:, 1].min() - 1, X_train_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))

Z = svm_model.predict(pca.inverse_transform(np.c_[xx.ravel(), yy.ravel()]))
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')

plt.colorbar(scatter)
plt.savefig(os.path.join(f"{plot_dir}/model_training", "svm_decision_boundary.png"))
plt.show()


### Lưu mô hình SVM
Mô hình được lưu vào file `models/svm_model.pkl`.


In [ ]:
# Lưu mô hình SVM
joblib.dump(svm_model, svm_model_path)
print(f"Mô hình SVM đã được lưu tại: {svm_model_path}")

### Đánh giá mô hình

#### Ma trận nhầm lẫn (Confusion Matrix)

In [5]:
# Dự đoán trên tập test
y_test_pred = svm_model.predict(X_test_features)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix:\n", conf_matrix)

# Báo cáo chi tiết
print("Classification Report:\n", classification_report(y_test, y_test_pred))

disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=["Không Gian Lận", "Gian Lận"])
disp.plot(cmap='Blues', values_format='d')
plt.title("Ma Trận Nhầm Lẫn")
plt.savefig(os.path.join(f"{plot_dir}/evaluation", "confusion_matrix.png"))
plt.show()



Confusion Matrix:
 [[92749   104]
 [    0   938]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     92853
           1       0.90      1.00      0.95       938

    accuracy                           1.00     93791
   macro avg       0.95      1.00      0.97     93791
weighted avg       1.00      1.00      1.00     93791



#### Biểu đồ ROC

In [ ]:
# Tính ROC Curve
fpr, tpr, _ = roc_curve(y_test, svm_model.decision_function(X_test))
roc_auc = auc(fpr, tpr)

# Vẽ Biểu Đồ ROC
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.title("Biểu Đồ ROC")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.savefig(os.path.join(f"{plot_dir}/evaluation", "roc_curve.png"))
plt.show()